In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [6]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [7]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([1])


In [8]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


In [9]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [10]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [11]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.3510, -0.0426,  0.2315, -0.0629, -0.3459,  0.3328, -0.1781,  0.1270,
         -0.5704, -0.0943,  0.5755, -0.5987,  0.5542,  0.1910,  0.4595, -0.3207,
          0.3848, -0.1329, -0.2397,  0.4325],
        [-0.0884, -0.4088, -0.0431, -0.1226, -0.3084,  0.2824, -0.2036,  0.0282,
         -0.1569, -0.0265,  0.6668, -0.1069,  0.8200, -0.1302, -0.0393, -0.2778,
          0.2437, -0.0009, -0.0236,  0.0891],
        [-0.0250, -0.0775,  0.0411, -0.1589, -0.2437,  0.1863, -0.1202, -0.0485,
         -0.3509, -0.3361,  0.6870, -0.6081,  0.6287,  0.2603,  0.2793, -0.5514,
          0.1121,  0.1438, -0.3484,  0.2417]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.2315, 0.0000, 0.0000, 0.3328, 0.0000, 0.1270, 0.0000,
         0.0000, 0.5755, 0.0000, 0.5542, 0.1910, 0.4595, 0.0000, 0.3848, 0.0000,
         0.0000, 0.4325],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.2824, 0.0000, 0.0282, 0.0000,
         0.0000, 0.6668, 0.0000, 0.8200, 0.0000, 0.00

In [12]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

In [13]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [14]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0244, -0.0174,  0.0339,  ...,  0.0334,  0.0273,  0.0171],
        [ 0.0063,  0.0247, -0.0281,  ...,  0.0022,  0.0160,  0.0175]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0249, -0.0335], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0222, -0.0087,  0.0346,  ..., -0.0179,  0.0120, -0.0339],
        [-0.0013, -0.0033,  0.0068,  ..., -0.0165, -0.0110, -0.0147]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | 